In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
class myCallback (tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98 and logs.get('val_accuracy')>0.85):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
def create_model():
    mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
    mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))
    mobile_net_layers.trainable = False

    model = tf.keras.Sequential([
        mobile_net_layers,
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(9,activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data/training/'
    validation_dir = '../data/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    history = model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=callback)
    return model



In [5]:
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 9)                 11529     
                                                                 
Total params: 2,269,513
Trainable params: 11,529
Non-trainable params: 2,257,984
_________________________________________________________________
Found 317 images belonging to 9 classes.
Found 82 images belonging to 9 classes.
Epoch 1/100
64/64 [==============================] - 17s 207ms/step - loss: 2.0015 - accuracy: 0.3060 - val_loss: 1.0877 - val_accuracy: 0.6829
Epoch 2/100
64/64 [==============================] - 8s 129ms/step - loss: 0.931

In [14]:
# Specify the local image path
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

local_image_path = '../data/validation/Tempe Goreng/tempe-goreng-masako-foto-resep-utama.jpg'

# Make predictions for the local image
img = image.load_img(local_image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = model.predict(img_array)
print(predictions)

predicted_class = np.argmax(predictions)

print(f"The predicted class is: {predicted_class}")

1/1 [==============================] - 0s 44ms/step
[[2.8212075e-03 1.5857372e-02 7.8405184e-04 2.2272839e-05 2.7531825e-03
  4.9960145e-01 4.3740121e-01 4.0057961e-02 7.0131844e-04]]
The predicted class is: 5
